# SakeMaker Pipelines

This notebook uses the [penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data).

<img src='https://imgur.com/orZWHly.png' alt='Penguins dataset' width="1024">

Let's make sure we are running the latest version of the SakeMaker's SDK. Restart the notebook after you upgrade the library.

In [4]:
!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker
!pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.22 requires botocore==1.23.22, but you have botocore 1.29.109 which is incompatible.
awscli 1.22.22 requires s3transfer<0.6.0,>=0.5.0, but you have s3transfer 0.6.0 which is incompatible.
Name: sagemaker
Version: 2.145.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, protobuf3-to-dict, PyYAML, schema, smdebug-rulesconfig
Required-by: 


Look at:

* Caching Pipeline Steps https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html

In [20]:
import os
import sagemaker
import numpy as np
import boto3
import json
import pandas as pd
import numpy as np
import urllib.request
import argparse
import logging

from botocore.exceptions import ClientError
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import FileSystemInput
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker import ModelPackage


role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Step 1 - Downloading the Dataset

The first step is to download the dataset and store it in S3.

In [3]:
DATA_FILEPATH = "penguins/data.csv"
S3_FILEPATH = "s3://mlschool/penguins"

# Download the official Penguins dataset and store it locally.
urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

# Upload the dataset to S3. We need to do this to make it available to 
# the preprocessing step.
INPUT_DATA_URI = sagemaker.s3.S3Uploader.upload(
    local_path=DATA_FILEPATH, 
    desired_s3_uri=S3_FILEPATH,
)

INPUT_DATA_URI

's3://mlschool/penguins/data.csv'

## Step 2 - Configuring the Pipeline

These are the parameters that will be used by the Pipeline:

* `dataset_location`: This parameter represents the location of the dataset in S3. We'll use this parameter during the Preprocessing Step to perform feature engineering on the original data.

In [4]:
dataset_location = ParameterString(
    name="dataset_location",
    default_value=INPUT_DATA_URI,
)

# We need to predefine the location where the preprocessing step will be
# storing the dataset splits to avoid SageMaker from appending a timestamp
# to their auto-generated location. If we let SageMaker use a timestamp, 
# we can't cache this step.
preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f'{S3_FILEPATH}/preprocessing',
)


# This represents the approval status of a new trained model in the registry.
model_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="Approved"
)

# This property represents the minimum accuracy that the model should
# reach in order for it to be registered.
accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.75
)


# We can use this cache configuration to cache individual pipeline steps.
# If we enable it, SageMaker will reuse the output of a previous successful
# run.
cache_config = CacheConfig(enable_caching=True, expire_after="5d")


## Step 3 - Preprocessing the Dataset

This script is responsible from doing feature engineering on the original dataset and spliting the data into train, validation, and a test set.

In [5]:
%%writefile penguins/preprocessor.py

import os
import numpy as np
import pandas as pd
import tempfile

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler


BASE_DIR = "/opt/ml/processing"
INPUT_PATH = Path(BASE_DIR) / "input"
DATA_FILEPATH = INPUT_PATH / "data.csv"


def preprocess(base_dir, data_filepath):
    df = pd.read_csv(data_filepath)
    
    numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]

    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    categorical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
            ("categorical", categorical_preprocessor, ["island"]),
        ]
    )
    

    X = df.drop(["sex"], axis=1)
    columns = list(X.columns)
    
    X = X.to_numpy()
    
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7 * len(X)), int(.85 * len(X))])
    
    X_train = pd.DataFrame(train, columns=columns)
    X_validation = pd.DataFrame(validation, columns=columns)
    X_test = pd.DataFrame(test, columns=columns)
    
    y_train = X_train.species
    y_validation = X_validation.species
    y_test = X_test.species

    X_train.drop(["species"], axis=1, inplace=True)
    X_validation.drop(["species"], axis=1, inplace=True)
    X_test.drop(["species"], axis=1, inplace=True)
    
    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)

    label_encoder = LabelEncoder()

    y_train = label_encoder.fit_transform(y_train)
    y_validation = label_encoder.transform(y_validation)
    y_test = label_encoder.transform(y_test)
    
    
    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)
    
    train_path = Path(base_dir) / "train" 
    validation_path = Path(base_dir) / "validation" 
    test_path = Path(base_dir) / "test"
    
    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)
    
    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


if __name__ == "__main__":
    run(BASE_DIR, DATA_FILEPATH)


Overwriting penguins/preprocessor.py


We can now create a Processing Step to run our preprocessing script. We will use a Scikit-Learn processor.

The input of this step will be the dataset location, and the output will be the location of the three sets.

In [6]:
sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing-processor",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

preprocess_step = ProcessingStep(
    name="PenguinsPreprocessor",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
    ],
    code="penguins/preprocessor.py",
    cache_config=cache_config
)

## Step 4 - Training the Model

This script is responsible from training a simple neural network on the train data, validating the model, and saving it so we can later use it.

In [7]:
%%writefile penguins/train.py


import os
import argparse

import numpy as np
import pandas as pd
import random
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dropout


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

Overwriting penguins/train.py


The Training Step will use a TensorFlow estimator.

The inputs to this step are the location of the train and the validation sets.

In [8]:
hyperparameters = {
    "epochs": 50,
    "batch_size": 32
}

estimator = TensorFlow(
    entry_point="penguins/train.py",
    role=role,
    hyperparameters=hyperparameters,
    instance_type="ml.m5.large",
    instance_count=1,
    py_version="py37",
    framework_version="2.4",
    script_mode=True,
    volume_size=5,
    disable_profiler=True
)

training_step = TrainingStep(
    name="PenguinsTraining",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

## Step 5 - Evaluating the Model

This script is reponsible from loading the model and evaluating it on the test set. Before finishing, this script will create a file containing the evaluation report.

In [9]:
%%writefile penguins/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score


MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
OUTPUT_PATH = "/opt/ml/processing/evaluation/"


def evaluate(model_path, test_path, output_path):
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    dir_list = os.listdir(Path(model_path))
    print("model_path", dir_list)
    
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")
    
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH
    )

Overwriting penguins/evaluation.py


The Processing Step will use a Script Processor running TensorFlow.

The inputs to this step are the location of the saved model and the location of the test set. The output is the evaluation report.

In [10]:
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4",
    py_version="py37",
    image_scope="training",
    instance_type="ml.m5.large"
)

evaluation_script_processor = ScriptProcessor(
    base_job_name="penguins-evaluation-processor",
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

evaluation_step = ProcessingStep(
    name="PenguinsEvaluation",
    processor=evaluation_script_processor,
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="penguins/evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config
)

## Step 6 - Registering the Model

In [11]:
model_package_group_name = "PenguinsModelPackage"

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=f"{evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']}/evaluation.json",
        content_type="application/json",
    )
)

image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4",
    py_version="py37",
    image_scope="inference",
    instance_type="ml.m5.large"
)

model = Model(
    image_uri=image_uri,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=PipelineSession(),
    role=role,
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
    approval_status=model_approval_status,
)

register_model_step = ModelStep(
   name="PenguinsCreateModel",
   step_args=register_args,
)

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:270: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Step 7 - Verifying the Model's Accuracy

We want to register the model only if their accuracy is above a predefined threshold. We can use a Condition Step to accomplish this.

In [12]:
condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

accuracy_condition_step = ConditionStep(
    name="PenguinsAccuracyCondition",
    conditions=[condition_gte],
    if_steps=[register_model_step],
    else_steps=[], 
)

## Step 8 - Running the Pipeline

We can now define and run the pipeline.

In [13]:
pipeline = Pipeline(
    name="PenguinsPipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        training_step, 
        evaluation_step,
        accuracy_condition_step
    ],
)

In [14]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.


## Step 9 - Deploying the Model

In [49]:
sagemaker_client = boto3.client("sagemaker")


def get_latest_approved_model_package(model_package_group_name):
    """
    Returns the latest approved model package registered under the 
    supplied model package group.
    """
    try:
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        while len(approved_packages) == 0 and "NextToken" in response:
            response = sagemaker_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        if len(approved_packages) == 0:
            error = f"No approved model pacakages for {model_package_group_name}"
            print(error)
            raise Exception(error)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        logger.info(f"Identified the latest approved model package: {model_package_arn}")
        return approved_packages[0]

    except ClientError as e:
        print(e.response["Error"]["Message"])
        raise Exception(e.response["Error"]["Message"])
        
        
def does_endpoint_exist(endpoint_name):
    """
    Returns whether the supplied endpoint already exists.
    """
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        return True
    except ClientError as e:
        return False
        

approved_model_package = get_latest_approved_model_package(model_package_group_name)
approved_model_package_arn = approved_model_package["ModelPackageArn"]

model_description = sagemaker_client.describe_model_package(
    ModelPackageName=approved_model_package_arn
)

model_description

{'ModelPackageGroupName': 'PenguinsModelPackage',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:325223348818:model-package/penguinsmodelpackage/1',
 'CreationTime': datetime.datetime(2023, 4, 8, 18, 48, 41, 648000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.4-cpu',
    'ImageDigest': 'sha256:9655faca3ac7c26a9583ad05a324ca7a58da4af1c14f3d2daae0d217688356a9',
    'ModelDataUrl': 's3://sagemaker-us-east-1-325223348818/pipelines-uuwrvnosz7n5-PenguinsTraining-ojUeWdBAwE/output/model.tar.gz',
    'Environment': {}}],
  'SupportedTransformInstanceTypes': ['ml.m5.large'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m5.large'],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': Fals

In [46]:
model_package = ModelPackage(
    role=role, 
    model_package_arn=approved_model_package_arn, 
    sagemaker_session=sagemaker_session
)

endpoint_name = "penguins-endpoint"

if not does_endpoint_exist(endpoint_name):
    model_package.deploy(
        initial_instance_count=1, 
        instance_type="ml.m5.large", 
        endpoint_name=endpoint_name
    )

## Step 10 - Testing the Model Endpoint

In [50]:
predictor = Predictor(endpoint_name=endpoint_name)

payload = "0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0"
p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})

predictions = json.loads(p.decode("utf-8"))["predictions"]

print(f"Prediction: {np.argmax(predictions)}")

Prediction: 2


## Step 11 - Clean up

In [55]:
for model_package in sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
    print(f"Deleting {model_package['ModelPackageArn']}")
    sagemaker_client.delete_model_package(ModelPackageName=model_package["ModelPackageArn"])

    
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)
predictor.delete_endpoint()
pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/penguinspipeline',
 'ResponseMetadata': {'RequestId': '533f9d2b-1aa8-4545-a30e-c98f4bd9862d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '533f9d2b-1aa8-4545-a30e-c98f4bd9862d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Sat, 08 Apr 2023 20:40:25 GMT'},
  'RetryAttempts': 0}}